In [1]:
import pandas as pd

In [ ]:
ls data

# ad.csv 크롤링 데이터
- `ad.csv`에서 `content_id` 기준으로 크롤링한 `ad_crawling.csv` 병합

In [2]:
ad_crawling = pd.read_csv('data/ad_new1.csv')
ad_df = pd.read_csv('data/ad.csv')

In [3]:
ad_df = pd.concat([ad_df, ad_crawling], axis=1)
ad_df = ad_df.drop('id', axis=1)
ad_df.head(2)

,content_id,user_id,name,keyword,price,flag_used,category_id_1,category_id_2,category_id_3,emergency_cnt,comment_cnt,interest,pfavcnt,b_pay,category,delivery_fee,place,status,text
0,157826057,114088,인텔 4세대 i5-4670 사무용 컴퓨터 팝니다.,"사무용컴퓨터,사무용PC,중고사무용컴퓨터,중고사무용PC,사무용중고컴퓨터",250000,1,600,600100,600100006,0,0,714,4,1,NaN,배송비포함,서울특별시 용산구 한강로동,판매중,"상품정보\n[ 컴퓨터 상세 사양 ] \n(SSD, 케이스만 신품, 나머지 중고)\n..."
1,162431268,3247363,갤럭시노트10+ 512GB 글로우 (무잔상) ! 0821,"갤럭시노트10,노트10플러스,노트10플러스중고,노트10플러스공기,노트10플러스글로",439000,1,600,600700,600700001,0,0,39,2,0,NaN,NaN,NaN,삭제됨,NaN


In [4]:
ad_crawling['status'].unique()

array(['판매중', '삭제됨', '거래 완료'], dtype=object)

- 저장

In [5]:
#ad_df.to_csv('data/ad_new.csv', index=False)

# 병합

In [6]:
impression_df = pd.read_csv('data/impression_log.csv')
view_df = pd.read_csv('data/view_log.csv')

#dataset_df = pd.read_csv('data/dataset.csv')

## impression + view
- label : 노출 - 0 , view - 1로 처리>\?

In [7]:
impression_df = impression_df.drop_duplicates()
view_df = view_df.drop_duplicates()

In [8]:
# impression 데이터에 log가 없는 view들의 id
no_imp_id = list(set(view_df['imp_id']) - set(impression_df['imp_id']))

In [9]:
# impression 데이터에 log가 없는 view들 모음
no_imp_view = view_df.set_index('imp_id').loc[no_imp_id].reset_index()

In [10]:
# 기존 view_df에서 impression과 겹치지 않는 id 제거
view_df = view_df.set_index('imp_id').drop(no_imp_id).reset_index()

In [11]:
view_df

,imp_id,server_time_kst,bid_price,device_type
0,99d1612dded122458e99,2021-08-31T16:56:46.013+09:00,50,a
1,998c612ddab9008cceee,2021-08-31T16:32:01.533+09:00,385,a
2,998c612dd77f007f2d67,2021-08-31T16:17:26.423+09:00,50,a
3,971a612ddff00eb50a47,2021-08-31T16:53:48.436+09:00,70,a
4,971a612ddd140ea93f30,2021-08-31T16:41:29.857+09:00,155,a
...,...,...,...,...
26478,9919612d14c1203c9bf0,2021-08-31T02:26:35.932+09:00,50,a
26479,97f9612d1250145c2e00,2021-08-31T02:16:06.295+09:00,60,a
26480,99d1612d19e220565606,2021-08-31T02:48:24.894+09:00,60,a
26481,99d1612d12bf2044dde5,2021-08-31T02:18:03.444+09:00,80,a


In [12]:
# impression_df의 imp_id, user_id 기준으로 view_df의 view들에 user_id 붙여줌
imp_user = impression_df[['imp_id', 'user_id', 'content_id']]
view_df = view_df.merge(imp_user, 'left').drop_duplicates()

### 라벨 생성

- 광고를 클릭했으면 1, 노출만 됐으면 0을 부여해서 병합

In [14]:
view_df['label'] = 1
view_df

,imp_id,server_time_kst,bid_price,device_type,user_id,content_id,label
0,99d1612dded122458e99,2021-08-31T16:56:46.013+09:00,50,a,1728109,137078818,1
1,998c612ddab9008cceee,2021-08-31T16:32:01.533+09:00,385,a,77328955,132419409,1
2,998c612dd77f007f2d67,2021-08-31T16:17:26.423+09:00,50,a,75559871,162053601,1
3,971a612ddff00eb50a47,2021-08-31T16:53:48.436+09:00,70,a,9461170,161384072,1
4,971a612ddd140ea93f30,2021-08-31T16:41:29.857+09:00,155,a,10809468,162732366,1
...,...,...,...,...,...,...,...
26511,9919612d14c1203c9bf0,2021-08-31T02:26:35.932+09:00,50,a,74774702,163161249,1
26512,97f9612d1250145c2e00,2021-08-31T02:16:06.295+09:00,60,a,13008046,162188680,1
26513,99d1612d19e220565606,2021-08-31T02:48:24.894+09:00,60,a,76011534,161297627,1
26514,99d1612d12bf2044dde5,2021-08-31T02:18:03.444+09:00,80,a,1581224,142540474,1


In [15]:
impression_df['label'] = 0
impression_df

,imp_id,content_id,server_time_kst,bid_price,user_id,device_type,label
0,97cf612d3a7008353958,161592430,2021-08-31T05:07:20.945+09:00,50,11555138,a,0
1,99d1612d39112085014f,162892746,2021-08-31T05:02:38.277+09:00,50,1310312,a,0
2,97ca612d461111487741,162806628,2021-08-31T05:57:03.384+09:00,55,76429349,a,0
3,97f9612d3995149e6b11,161593862,2021-08-31T05:03:49.145+09:00,270,74800572,a,0
4,97f9612d3a22149eef25,161587624,2021-08-31T05:06:26.494+09:00,50,4454345,a,0
...,...,...,...,...,...,...,...
852582,9919612e429a24183e0d,163247160,2021-08-31T23:54:23.433+09:00,50,7660589,a,0
852583,998c612e425a027cdb47,78159183,2021-08-31T23:54:17.029+09:00,65,3735841,a,0
852584,9919612e429724183042,152627901,2021-08-31T23:54:24.948+09:00,70,9140358,a,0
852585,97f9612e4295183f6e39,160836652,2021-08-31T23:54:20.398+09:00,165,1668136,a,0


In [16]:
our_dataset = impression_df.append(view_df)
our_dataset

D:\anaconda\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,bid_price,content_id,device_type,imp_id,label,server_time_kst,user_id
0,50,161592430,a,97cf612d3a7008353958,0,2021-08-31T05:07:20.945+09:00,11555138
1,50,162892746,a,99d1612d39112085014f,0,2021-08-31T05:02:38.277+09:00,1310312
2,55,162806628,a,97ca612d461111487741,0,2021-08-31T05:57:03.384+09:00,76429349
3,270,161593862,a,97f9612d3995149e6b11,0,2021-08-31T05:03:49.145+09:00,74800572
4,50,161587624,a,97f9612d3a22149eef25,0,2021-08-31T05:06:26.494+09:00,4454345
...,...,...,...,...,...,...,...
26511,50,163161249,a,9919612d14c1203c9bf0,1,2021-08-31T02:26:35.932+09:00,74774702
26512,60,162188680,a,97f9612d1250145c2e00,1,2021-08-31T02:16:06.295+09:00,13008046
26513,60,161297627,a,99d1612d19e220565606,1,2021-08-31T02:48:24.894+09:00,76011534
26514,80,142540474,a,99d1612d12bf2044dde5,1,2021-08-31T02:18:03.444+09:00,1581224


# 나머지 feature들 붙이기

In [17]:
ad_df = pd.read_csv('data/ad_new.csv')
advertiser_df = pd.read_csv('data/advertiser_title.csv')
viewer_df = pd.read_csv('data/viewer.csv')

### Ad.csv
- 크롤링한 데이터를 포함해서 컬럼명 조정 후 병합 진행

In [18]:
ad_dict = {'user_id': 'advertiser_id', 'name': 'content_name', 'keyword': 'content_keyword', 
           'price': 'content_price', 'flag_used': 'content_used', 
           'category_id_1': 'content_cat_1', 'category_id_2': 'content_cat_2', 'category_id_3': 'content_cat_3', 
          'emergency_cnt': 'content_emergency_count', 'comment_cnt': 'content_comment_count',
          'interest': 'content_views', 'pfavcnt': 'content_likes', 
          'status': 'content_status', 'b_pay': 'content_b_pay', 'place': 'content_place',
          'text': 'content_text', 'delivery_fee': 'content_delivery_fee'}
ad_df = ad_df.rename(ad_dict, axis=1)
ad_df.head(2)

,content_id,advertiser_id,content_name,content_keyword,content_price,content_used,content_cat_1,content_cat_2,content_cat_3,content_emergency_count,content_comment_count,content_views,content_likes,content_b_pay,category,content_delivery_fee,content_place,content_status,content_text
0,157826057,114088,인텔 4세대 i5-4670 사무용 컴퓨터 팝니다.,"사무용컴퓨터,사무용PC,중고사무용컴퓨터,중고사무용PC,사무용중고컴퓨터",250000,1,600,600100,600100006,0,0,714,4,1,NaN,배송비포함,서울특별시 용산구 한강로동,판매중,"상품정보\n[ 컴퓨터 상세 사양 ] \n(SSD, 케이스만 신품, 나머지 중고)\n..."
1,162431268,3247363,갤럭시노트10+ 512GB 글로우 (무잔상) ! 0821,"갤럭시노트10,노트10플러스,노트10플러스중고,노트10플러스공기,노트10플러스글로",439000,1,600,600700,600700001,0,0,39,2,0,NaN,NaN,NaN,삭제됨,NaN


- 일단 카테고리 대분류만 한글로 변경

In [19]:
cat2kor_1 = {220: '지역 서비스', 310: '여성 의류', 320: '남성 의류', 
             400: '남성 악세사리', 405: '신발', 410: '미용품', 420: '시계/쥬얼리', 430: '가방',
            500: '유아동/출산', 600: '전자제품', 700: '스포츠/레저', 750: '차량/오토바이', 800: '생활/가공식품',
            810: '가구/인테리어', 900: '도서/티켓/문구', 910: '스타굿즈', 920: '음반/악기', 930: '키덜트',
             980: '반려동물용품', 990: '예술/희귀/수집품',999: '기타'
            }
ad_df['content_cat_1'] = ad_df['content_cat_1'].apply(lambda x: cat2kor_1[x])
ad_df.head()

,content_id,advertiser_id,content_name,content_keyword,content_price,content_used,content_cat_1,content_cat_2,content_cat_3,content_emergency_count,content_comment_count,content_views,content_likes,content_b_pay,category,content_delivery_fee,content_place,content_status,content_text
0,157826057,114088,인텔 4세대 i5-4670 사무용 컴퓨터 팝니다.,"사무용컴퓨터,사무용PC,중고사무용컴퓨터,중고사무용PC,사무용중고컴퓨터",250000,1,전자제품,600100,600100006,0,0,714,4,1,NaN,배송비포함,서울특별시 용산구 한강로동,판매중,"상품정보\n[ 컴퓨터 상세 사양 ] \n(SSD, 케이스만 신품, 나머지 중고)\n..."
1,162431268,3247363,갤럭시노트10+ 512GB 글로우 (무잔상) ! 0821,"갤럭시노트10,노트10플러스,노트10플러스중고,노트10플러스공기,노트10플러스글로",439000,1,전자제품,600700,600700001,0,0,39,2,0,NaN,NaN,NaN,삭제됨,NaN
2,162806215,12851816,#한정판매#갤럭시S21 256GB 화이트 SS급 중고폰공기계[75333],"갤럭시S21,갤럭시S21화이트,S21화이트",620000,1,전자제품,600700,600700001,0,0,145,5,1,NaN,배송비포함,서울특별시 강남구 논현2동,판매중,"상품정보\n★ 번개장터 구매고객 특별이벤트 ★\n\n1,구입후 중고폰 100일 제품..."
3,152625380,2347538,#당일발송#갤럭시S9플러스 64G 대량판매 등급별초특가!,"S9플러스,S9플러스중고,갤럭시S9플러스",195000,1,전자제품,600700,600700001,0,0,234,8,0,NaN,NaN,NaN,삭제됨,NaN
4,161572473,12927760,포터2카고 중고화물차 팝니다.,"포터2,포터중고차,포터2카고,포터2중고,포터2카고중고",9800000,1,차량/오토바이,750200,750200700,0,0,388,4,0,NaN,NaN,NaN,삭제됨,NaN


- 병합

In [20]:
our_dataset = our_dataset.merge(ad_df, on='content_id')
our_dataset.head(2)

,bid_price,content_id,device_type,imp_id,label,server_time_kst,user_id,advertiser_id,content_name,content_keyword,...,content_emergency_count,content_comment_count,content_views,content_likes,content_b_pay,category,content_delivery_fee,content_place,content_status,content_text
0,50,162892746,a,99d1612d39112085014f,0,2021-08-31T05:02:38.277+09:00,1310312,12851816,#한정판매#갤럭시Z플립2 그레이 256GB 중고폰 공기계[26583],"갤럭시Z플립2,Z플립2그레이,Z플립2",...,0,0,353,9,0,NaN,배송비 별도,강남역,판매중,상품정보\n안녕하세요^^ 지상최고라고 감히 자부하겠습니다♡ 주름..이거 시술로만 가...
1,50,162892746,a,971a612dca900e618ff1,0,2021-08-31T15:22:15.629+09:00,6637605,12851816,#한정판매#갤럭시Z플립2 그레이 256GB 중고폰 공기계[26583],"갤럭시Z플립2,Z플립2그레이,Z플립2",...,0,0,353,9,0,NaN,배송비 별도,강남역,판매중,상품정보\n안녕하세요^^ 지상최고라고 감히 자부하겠습니다♡ 주름..이거 시술로만 가...


### Advertiser.csv

- 컬럼명 변경, 중복 제거

In [21]:
adv_name_dict = {'user_id': 'advertiser_id','comment_count': 'adv_comment_count', 'follower_count' : 'drop', 'pay_count': 'adv_pay_count', 
            'parcel_post_count': 'adv_parcel_post_count', 'transfer_count': 'adv_transfer_count', 'chat_count': 'adv_chat_count',
                'grade': 'adv_grade', 'favorite_count': 'adv_follower_count', 'review_count': 'adv_review_count', 'interest': 'adv_views',
                'title': 'adv_title', 'item_count': 'adv_item_count'}
advertiser_df = advertiser_df.rename(adv_name_dict, axis=1)
advertiser_df = advertiser_df.drop_duplicates()
advertiser_df

,advertiser_id,adv_follower_count,adv_grade,adv_item_count,adv_views,adv_review_count,adv_comment_count,drop,adv_pay_count,adv_parcel_post_count,adv_transfer_count,adv_chat_count,adv_title
0,2487730,1899,7673,431,31681,775,529,0,575,0,35,130,이수컴
1,1462217,799,5520,433,16794,570,250,0,942,0,57,11,최저가전
2,6972746,3692,7526,2530,36944,776,828,0,325,0,10,13,북부통신
3,1358510,1228,2106,100,27142,223,431,0,232,9,1,4,바이크빌드
5,5051200,254,692,77,6462,74,0,0,196,8,40,0,스마일Tool
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,75231266,28,38,193,69,5,0,0,11,1,0,2,그림겔러리정리
1513,5332936,2,51,12,289,5,1,0,1,0,0,0,현규대장
1514,8173665,4,10,16,26,1,5,0,2,0,0,0,아리아리12
1524,2294187,462,3720,833,1228,376,0,0,186,1,6,60,저렴이중고가게


- 병합

In [23]:
our_dataset = our_dataset.merge(advertiser_df, on='advertiser_id')
our_dataset.head(2)

,bid_price,content_id,device_type,imp_id,label,server_time_kst,user_id,advertiser_id,content_name,content_keyword,...,adv_item_count,adv_views,adv_review_count,adv_comment_count,drop,adv_pay_count,adv_parcel_post_count,adv_transfer_count,adv_chat_count,adv_title
0,50,162892746,a,99d1612d39112085014f,0,2021-08-31T05:02:38.277+09:00,1310312,12851816,#한정판매#갤럭시Z플립2 그레이 256GB 중고폰 공기계[26583],"갤럭시Z플립2,Z플립2그레이,Z플립2",...,247,4525,542,739,0,496,0,0,11,AIROK
1,50,162892746,a,971a612dca900e618ff1,0,2021-08-31T15:22:15.629+09:00,6637605,12851816,#한정판매#갤럭시Z플립2 그레이 256GB 중고폰 공기계[26583],"갤럭시Z플립2,Z플립2그레이,Z플립2",...,247,4525,542,739,0,496,0,0,11,AIROK


### Viewer.csv

- 컬럼명 변경 및 중복 제거

In [22]:
viewer_name_dict = {'comment_count': 'viewer_comment_count', 'following_cnt' : 'viewer_following_count', 'pay_count': 'viewer_pay_count', 
            'parcel_post_count': 'viewer_parcel_post_count', 'transfer_count': 'viewer_transfer_count', 'chat_count': 'viewer_chat_count',
                   'gender': 'viewer_gender', 'age': 'viewer_age'}
viewer_df = viewer_df.rename(viewer_name_dict, axis=1)
viewer_df = viewer_df.drop_duplicates()
viewer_df

,user_id,viewer_gender,viewer_age,viewer_following_count,viewer_pay_count,viewer_parcel_post_count,viewer_transfer_count,viewer_chat_count
0,513808,2,48,0,0,0,0,0
1,10764680,1,25,0,0,0,0,0
2,516424,0,0,1,0,0,0,0
3,10248440,1,36,14,0,0,0,0
4,9741736,1,30,2,0,0,0,0
...,...,...,...,...,...,...,...,...
56376,77099027,2,60,1,0,0,0,1
56377,77172013,1,47,17,0,0,0,0
56378,77220633,2,51,4,0,0,0,0
56379,77270759,1,51,0,0,0,0,0


- 병합

In [24]:
our_dataset = our_dataset.merge(viewer_df, on='user_id')
our_dataset.head(2)

,bid_price,content_id,device_type,imp_id,label,server_time_kst,user_id,advertiser_id,content_name,content_keyword,...,adv_transfer_count,adv_chat_count,adv_title,viewer_gender,viewer_age,viewer_following_count,viewer_pay_count,viewer_parcel_post_count,viewer_transfer_count,viewer_chat_count
0,50,162892746,a,99d1612d39112085014f,0,2021-08-31T05:02:38.277+09:00,1310312,12851816,#한정판매#갤럭시Z플립2 그레이 256GB 중고폰 공기계[26583],"갤럭시Z플립2,Z플립2그레이,Z플립2",...,0,11,AIROK,1,24,3,4,0,0,0
1,50,162892746,a,971a612d3e480cf3d178,0,2021-08-31T05:24:14.109+09:00,1310312,12851816,#한정판매#갤럭시Z플립2 그레이 256GB 중고폰 공기계[26583],"갤럭시Z플립2,Z플립2그레이,Z플립2",...,0,11,AIROK,1,24,3,4,0,0,0


# 데이터 정리 후 저장

- 필요 없는 컬럼 제거

In [26]:
our_dataset = our_dataset.drop('drop', axis=1)

In [44]:
our_dataset = our_dataset.drop('device_type', axis=1)

In [30]:
our_dataset = our_dataset.rename({'advertiser_id': 'adv_id'}, axis=1)

In [33]:
our_dataset = our_dataset.drop('category', axis=1)

- 이미지 url 생성

In [37]:
our_dataset['content_img_url'] = our_dataset['content_id'].apply(lambda x: f'https://media.bunjang.co.kr/product/{x}_...')

In [45]:
our_dataset.iloc[1]

bid_price                                                                  50
content_id                                                          162892746
imp_id                                                   971a612d3e480cf3d178
label                                                                       0
server_time_kst                                 2021-08-31T05:24:14.109+09:00
user_id                                                               1310312
adv_id                                                               12851816
content_name                           #한정판매#갤럭시Z플립2 그레이 256GB 중고폰 공기계[26583]
content_keyword                                          갤럭시Z플립2,Z플립2그레이,Z플립2
content_price                                                          620000
content_used                                                                1
content_cat_1                                                            전자제품
content_cat_2                                                   

- 용량 문제로 full/text제외 나눠서 저장

In [47]:
our_notext = our_dataset.drop(['content_name', 'content_keyword', 'content_text', 'content_img_url', 'adv_title'], axis=1)

In [48]:
our_dataset.to_csv('data/our_data_full.csv', index=False)
our_notext.to_csv('data/our_data_notext.csv', index=False)

In [4]:
our_dataset = pd.read_csv('data/our_data_full.csv')

## 노션 업로드용 데이터 정리 후 저장

In [24]:
our_dataset[our_dataset['content_status']!='삭제됨'].sort_values('imp_id').iloc[3:13][['imp_id', 'content_id', 'user_id', 'adv_id', 'label', 'bid_price', 'server_time_kst',
       'content_name', 'content_keyword', 'content_price',
       'content_used', 'content_cat_1', 'content_cat_2', 'content_cat_3',
       'content_emergency_count', 'content_comment_count', 'content_views',
       'content_likes', 'content_b_pay', 'content_delivery_fee',
       'content_place', 'content_status', 'content_text', 'content_img_url',
        'adv_follower_count', 'adv_grade', 'adv_item_count', 'adv_views', 'adv_review_count',
       'adv_comment_count', 'adv_pay_count', 'adv_parcel_post_count',
       'adv_transfer_count', 'adv_chat_count', 'adv_title', 'viewer_gender',
       'viewer_age', 'viewer_following_count', 'viewer_pay_count',
       'viewer_parcel_post_count', 'viewer_transfer_count',
       'viewer_chat_count']].to_csv('노션용.csv', index=False)

In [25]:
notion = pd.read_csv('노션용.csv')

In [26]:
notion.loc[2]

imp_id                                                   971a612cc3520b270da5
content_id                                                          161017095
user_id                                                              77223113
adv_id                                                               11031323
label                                                                       0
bid_price                                                                  55
server_time_kst                                 2021-08-31T00:04:50.258+09:00
content_name                                                               불상
content_keyword                                          중국도자기,수집품,불상,골동품,도자기
content_price                                                          350000
content_used                                                                1
content_cat_1                                                       예술/희귀/수집품
content_cat_2                                                   